In [2]:
pip install torch torchvision opencv-python transformers timm effdet pillow

Note: you may need to restart the kernel to use updated packages.


In [8]:
import cv2
import torch
import numpy as np
from PIL import Image
from transformers import ViTImageProcessor, ViTForImageClassification
from effdet import get_efficientdet_config, EfficientDet, DetBenchPredict
from timm.data import create_transform
from omegaconf import OmegaConf


In [11]:

class CatAnalysisSystem:
    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        print(f"Using device: {self.device}")
        
        # --- STAGE 1: LOAD EFFICIENTDET (Object Detection) ---
        print("Loading EfficientDet...")
        # ใช้ tf_efficientdet_d0 (รุ่นเล็กสุดแต่เร็ว) หรือเปลี่ยนเป็น d1-d7 ถ้าต้องการแม่นขึ้น
        self.det_config = get_efficientdet_config('tf_efficientdet_d0')
        self.det_net = EfficientDet(self.det_config, pretrained_backbone=False)
        checkpoint = torch.hub.load_state_dict_from_url(
            "https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_d0_34-f153e0cf.pth", 
            map_location=device
        )
        self.det_net.load_state_dict(checkpoint)
        # 1. ปลดล็อค Config ให้แก้ไขได้
        OmegaConf.set_readonly(self.det_config, False) 

        # 2. ตอนนี้จะแก้ไขค่าได้แล้ว ไม่ error
        self.det_config.num_classes = 90     
        self.det_config.image_size = [512, 512]
        self.det_model = DetBenchPredict(self.det_net)
        self.det_model.eval().to(self.device)

        # --- STAGE 2: LOAD VISION TRANSFORMER (Classification) ---
        print("Loading Vision Transformer (ViT)...")
        # ใช้ Pre-trained ImageNet (มีแมวหลายสายพันธุ์)
        self.vit_model_name = 'google/vit-base-patch16-224' 
        self.vit_processor = ViTImageProcessor.from_pretrained(self.vit_model_name)
        self.vit_model = ViTForImageClassification.from_pretrained(self.vit_model_name)
        self.vit_model.eval().to(self.device)

    def detect_cats(self, img_cv2, threshold=0.5):
        """Stage 1: Detect objects and filter only Cats"""
        # เตรียมภาพสำหรับ EfficientDet
        img_rgb = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)
        src_img = Image.fromarray(img_rgb)
        
        # Transform (Resize/Normalize) ตาม config ของ EfficientDet
        transform = create_transform(
            self.det_config.image_size, 
            mean=self.det_config.mean, 
            std=self.det_config.std
        )
        img_tensor = transform(src_img).unsqueeze(0).to(self.device)

        with torch.no_grad():
            output = self.det_model(img_tensor)

        # Output format: [batch, max_det, 6] -> (x_min, y_min, x_max, y_max, score, class)
        results = output.cpu().numpy()[0]

        cat_boxes = []
        # COCO Dataset: Class ID 17 คือ Cat (ใน effdet index อาจจะเริ่มที่ 1 คือ Person, ดังนั้น Cat ~17)
        # หมายเหตุ: EfficientDet Pretrained ส่วนใหญ่ map COCO 90 classes. Cat ID มักจะเป็น 17.
        CAT_CLASS_ID = 17 
        
        for res in results:
            xmin, ymin, xmax, ymax, score, class_id = res
            if score > threshold and int(class_id) == CAT_CLASS_ID:
                # แปลงพิกัดกลับไปเป็นขนาดภาพจริง (เนื่องจาก input ถูก resize)
                h_orig, w_orig = img_cv2.shape[:2]
                h_model, w_model = self.det_config.image_size
                
                scale_x = w_orig / w_model
                scale_y = h_orig / h_model
                
                # ถ้า transform มีการ pad ต้องคำนวณละเอียดกว่านี้ แต่นี่คือแบบคร่าวๆ
                # เพื่อความแม่นยำสูงสุดควรใช้ transform ย้อนกลับ แต่เพื่อความง่ายใช้ ratio
                
                cat_boxes.append([
                    int(xmin * scale_x), int(ymin * scale_y), 
                    int(xmax * scale_x), int(ymax * scale_y), 
                    score
                ])
                
        return cat_boxes

    def classify_breed(self, crop_img_cv2):
        """Stage 2: Classify breed using ViT"""
        if crop_img_cv2.size == 0: return "Unknown", 0.0
        
        img_rgb = cv2.cvtColor(crop_img_cv2, cv2.COLOR_BGR2RGB)
        inputs = self.vit_processor(images=img_rgb, return_tensors="pt").to(self.device)

        with torch.no_grad():
            outputs = self.vit_model(**inputs)
            logits = outputs.logits
            probs = logits.softmax(dim=1)
            top_prob, top_idx = probs.max(1)
        
        # ดึงชื่อ Label จาก ViT config
        breed_name = self.vit_model.config.id2label[top_idx.item()]
        # ตัดคำให้สั้นลง (เช่น "tabby, tabby cat" -> "tabby")
        breed_name = breed_name.split(',')[0] 
        
        return breed_name, top_prob.item()

    def run(self, image_path):
        img = cv2.imread(image_path)
        if img is None:
            print("Image not found")
            return

        # 1. Detection
        boxes = self.detect_cats(img)
        print(f"Found {len(boxes)} cats.")

        for box in boxes:
            xmin, ymin, xmax, ymax, score = box
            
            # กัน Error กรณีพิกัดออกนอกภาพ
            xmin, ymin = max(0, xmin), max(0, ymin)
            xmax, ymax = min(img.shape[1], xmax), min(img.shape[0], ymax)

            # 2. Crop Image
            cat_crop = img[ymin:ymax, xmin:xmax]
            
            # 3. Classification
            breed, conf = self.classify_breed(cat_crop)
            print(f"Cat at [{xmin},{ymin}] is likely: {breed} ({conf:.2f})")

            # Draw
            cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            label = f"{breed}: {conf:.2f}"
            cv2.putText(img, label, (xmin, ymin - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Show result
        cv2.imshow("Two-Stage Cat Analysis", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


In [ ]:
# --- วิธีใช้งาน ---
if __name__ == "__main__":
    # ใส่ path รูปแมวของคุณที่นี่
    image_path = "C:/Users/Advice IT/MeowScannerWeb/cat.jpeg" 
    
    system = CatAnalysisSystem()
    system.run(image_path)

Using device: cpu
Loading EfficientDet...
Loading Vision Transformer (ViT)...


c:\anaconda\envs\meowscanenv\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Advice IT\.cache\huggingface\hub\models--google--vit-base-patch16-224. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
